In [ ]:
#!matplotlib inline
import numpy as np
import torch
torch.set_printoptions(edgeitems=2, linewidth=75)

In [2]:
t_c = [0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c = torch.tensor(t_c)
t_u = torch.tensor(t_u)

* 모델 생성

In [3]:
def model(t_u, w, b):
    return w * t_u + b

* 손실 함수 설정 (mean_squared_error)

In [4]:
def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c)**2
    return squared_diffs.mean()

In [5]:
w = torch.ones(())
b = torch.zeros(())

t_p = model(t_u, w , b)
t_p

tensor([35.7000, 55.9000, 58.2000, 81.9000, 56.3000, 48.9000, 33.9000,
        21.8000, 48.4000, 60.4000, 68.4000])

In [6]:
loss = loss_fn(t_p, t_c)
loss

tensor(1763.8848)

# 손실 줄이기

* 편미분 공식

In [7]:
# bias 값
delta = 0.1

loss_rate_of_change_w = (loss_fn(model(t_u, w + delta,b), t_c) - loss_fn(model(t_u, w + delta,b), t_c)) / (2.0 * delta) 

In [8]:
import math
learning_rate = 1e-2

w = w - learning_rate * loss_rate_of_change_w
w

tensor(1.)

In [9]:
loss_rate_of_change_b = (loss_fn(model(t_u, w, b + delta), t_c) - loss_fn(model(t_u, w,b - delta), t_c)) / (2.0 * delta) 
b = b - learning_rate * loss_rate_of_change_b
b


tensor(-0.8260)

* MSE 공식

In [10]:
def loss_fn(t_p , t_c):
    squared_diffs = (t_p - t_c) ** 2
    return squared_diffs.mean()

### MSE를 미분하여 모델 파라미터의 기울기를 구할 수 있다.

In [11]:
def dloss_fn(t_p , t_c):
    dsq_diff = 2 * (t_p - t_c)  / t_p.size(0)
    return dsq_diff

# 모델에 미분 적용하기

In [12]:
# 모델 생성
def model(t_u , w , b):
    return t_u * w + b

* 가중치 w 에 대한 편미분
    - t_u * w  + b 공식에서 w 에 대한 편미분을 진행했으므로 t_u 만 남는다

In [13]:
def dmodel_dw(t_u, w , b):
    return t_u


* 편향 b에 대한 편미분
    - t_u * w 공식에서 b에 대한 편미분을 진행햤는데 b는 상수이므로 1.0 만 남는다 

In [14]:
def dmodel_db(t_u, w , b):
    return 1.0

### 경사 함수 정의
- 모델이 학습할 때 필요한 기울기 계산
- 모델의 파라미터에 대한 손실 함수의 기울기를 계산
- 미분 연쇄 법칙이 적용
    - 복합함수의 미분을 구할 때 각 함수의 미분을 곱하는 방법
    - 여기서는 weight 와 bias 가 기울기(mse의 미분값)와 곱해진다

In [15]:
def grad_fn(t_u ,t_c, t_p, w , b):
    dloss_dtp = dloss_fn(t_p, t_c)
    dloss_dw = dloss_dtp * dmodel_dw(t_u, w , b)
    dloss_db = dloss_dtp *  dmodel_db(t_u, w , b)
    return torch.stack([dloss_dw.sum(), dloss_db.sum()])


## 모델 적합을 위하여 반복

In [16]:
def training_loop(n_epochs , learning_rate , params, t_u, t_c):
    for epoch in range(1, n_epochs + 1):
        w , b = params
        
        t_p = model(t_u , w , b) #순방향 전달
        loss = loss_fn(t_p , t_c) #Mean squeared Error
        grad = grad_fn(t_u ,t_c, t_p, w , b) #역방향 전달

        # 파라미터 조정
        params = params - learning_rate * grad

        print(f'Epoch : {epoch}  Loss : {loss:.2f}  Grad : {grad}')
    return params
        

In [17]:
'''
def training_loop(n_epochs, learning_rate, params, t_u, t_c,
                  print_params=True):
    for epoch in range(1, n_epochs + 1):
        w, b = params

        t_p = model(t_u, w, b)  # <1>
        loss = loss_fn(t_p, t_c)
        grad = grad_fn(t_u, t_c, t_p, w, b)  # <2>

        params = params - learning_rate * grad

        if epoch in {1, 2, 3, 10, 11, 99, 100, 4000, 5000}:  # <3>
            print('Epoch %d, Loss %f' % (epoch, float(loss)))
            if print_params:
                print('    Params:', params)
                print('    Grad:  ', grad)
        if epoch in {4, 12, 101}:
            print('...')

        if not torch.isfinite(loss).all():
            break  # <3>
            
    return params
'''

"\ndef training_loop(n_epochs, learning_rate, params, t_u, t_c,\n                  print_params=True):\n    for epoch in range(1, n_epochs + 1):\n        w, b = params\n\n        t_p = model(t_u, w, b)  # <1>\n        loss = loss_fn(t_p, t_c)\n        grad = grad_fn(t_u, t_c, t_p, w, b)  # <2>\n\n        params = params - learning_rate * grad\n\n        if epoch in {1, 2, 3, 10, 11, 99, 100, 4000, 5000}:  # <3>\n            print('Epoch %d, Loss %f' % (epoch, float(loss)))\n            if print_params:\n                print('    Params:', params)\n                print('    Grad:  ', grad)\n        if epoch in {4, 12, 101}:\n            print('...')\n\n        if not torch.isfinite(loss).all():\n            break  # <3>\n            \n    return params\n"

# 휸련 진행
- loss가 무한대가 되어버린다
- params 조정이 너무 크다는 신호
    - 값이 앞뒤로 진동하면서 조정 값이 점점 커진다
- 최적화는 불안정해지고 , 수렴이 아닌 발산 해버린다

In [ ]:
training_loop(
    n_epochs = 100 ,
    learning_rate = 1e-2 , 
    params = torch.tensor([1.0 ,0.0]), 
    t_u = t_u, 
    t_c = t_c)

## 해결방법
- learning_rate 를 줄인다

In [ ]:
import numpy as no
training_loop(
    n_epochs = 100 ,
    learning_rate = 1e-4 , 
    params = torch.tensor([1.0 ,0.0]), 
    t_u = t_u, 
    t_c = t_c)

* 위의 코드에서 가중치에 대한 기울기는 편향에 대한 기울기보다 약 50배정도 큼
    - 가중치와 편향값의 범위가 다르다는 뜻
    - 각각 범위를 조정하지 않고 t_u 값을 정규화 해준다

In [20]:
t_un = t_u * 0.1

In [ ]:
import numpy as no
training_loop(
    n_epochs = 100 ,
    learning_rate = 1e-2 , 
    params = torch.tensor([1.0 ,0.0]), 
    t_u = t_un, 
    t_c = t_c)

* parmams 값의 변화량이 줄어들 때까지 학습

In [ ]:
import numpy as no
training_loop(
    n_epochs = 5000 ,
    learning_rate = 1e-2 , 
    params = torch.tensor([1.0 ,0.0]), 
    t_u = t_un, 
    t_c = t_c)

* 시각화

In [28]:
!which python
!python --version


/opt/anaconda3/bin/python
Python 3.12.2


In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

t_p = model(t_un, *params)  # <1>

fig = plt.figure(dpi=600)
plt.xlabel("Temperature (°Fahrenheit)")
plt.ylabel("Temperature (°Celsius)")
plt.plot(t_u.numpy(), t_p.detach().numpy()) # <2>
plt.plot(t_u.numpy(), t_c.numpy(), 'o')
plt.savefig("temp_unknown_plot.png", format="png")  # bookskip